Data Collection Part 1: Cleaning Up and Creating Fire Data

In [ ]:
import pandas as pd
import random
import numpy as np
import requests
#These "RAW" urls may require you to get you own "RAW" url because they stop working after some time (weird github thing)
url = 'https://raw.githubusercontent.com/UBC-CPEN291/project-team-corythosaurus/main/data/H_FIRE_PNT.csv?token=AQTQKXKXJO3RQOQTXVLK7UTATOHA4'
df = pd.read_csv(url, index_col=0)
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FIRE_YEAR,IGN_DATE,FIRE_CAUSE,FIRELABEL,FRCNTR,ZONE,FIRE_ID,FIRE_TYPE,GEO_DESC,LATITUDE,LONGITUDE,SIZE_HA,FCODE,SHAPE,OBJECTID,X,Y
FIRE_NO,,,,,,,,,,,,,,,,,
K30894,2001,2.001082e+13,Person,2001-K30894,5,3,500894,Nuisance Fire,Nielson Beach #1,51.0167,-119.0167,NaN,JA70003000,NaN,11402317,-119.016667,51.016667
V70397,2010,2.010071e+13,Person,2010-V70397,2,7,200397,Fire,Extension Road,49.1102,-123.9395,0.009,JA70003000,NaN,11402318,-123.939484,49.110249
K40261,2004,2.004070e+13,Lightning,2004-K40261,5,4,500261,Fire,Fintry,50.1537,-119.5490,0.009,JA70003000,NaN,11402319,-119.548950,50.153733
R50229,2019,2.019051e+13,Person,2019-R50229,3,5,300229,Fire,.6km south of Arbor FSR on Hwy 16,55.8015,-128.8971,0.009,JA70003000,NaN,11402320,-128.897117,55.801450
V81229,2009,2.009080e+13,Person,2009-V81229,2,8,201229,Nuisance Fire,Cumberland Dump,49.6422,-125.0558,0.000,JA70003000,NaN,11402321,-125.055783,49.642200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N10266,1999,1.999090e+07,Person,1999-N10266,6,1,266,Fire,FORT STEELE,49.4923,-115.4473,0.009,JA70003000,NaN,11579248,-115.447333,49.492333
N10267,1999,1.999081e+07,Lightning,1999-N10267,6,1,267,Fire,CAVEN CREEK,49.2630,-115.3325,0.200,JA70003000,NaN,11579249,-115.332500,49.263000
N10269,1999,1.999091e+07,Person,1999-N10269,6,1,269,Fire,Hidden Valley,49.4748,-115.7871,0.100,JA70003000,NaN,11579250,-115.787067,49.474767


In [ ]:
#Dropping features we dont need
df.drop(columns=['FIRE_YEAR', 'FIRELABEL', 'FRCNTR', 'ZONE', 'OBJECTID', 'FIRE_ID', 'GEO_DESC', 'FCODE', 'SHAPE', 'OBJECTID', 'X', 'Y'], inplace=True, errors='ignore')

#Making it so that data rows have valid SIZE_HA (fire size in hectares) and IGN_DATE (date of one row of data)
df['IGN_DATE'] = df['IGN_DATE'].div(1000000)
df = df[df['SIZE_HA'].notnull()]
df = df[df['IGN_DATE'].notnull()]

#Use data after 1970
df = df[df['IGN_DATE'] >= 1970]

#Group data that has one common feature: There is no fire
df_nofire = df[df['SIZE_HA'] == 0]
df_nofire = df_nofire.head(2500)

#Group data that has one common feature: There is fire (We define "there is fire" as there exists fire with SIZE_HA larger than 1.9)
df_yesfire = df[df['SIZE_HA'] > 1.9]

In [ ]:
total_nofire_list = []
#Create data for no fire (this is fine because the chances that a fire happening at a random location given random date is pretty small)
for i in range(2500):
  one_entry = []
  lat = random.uniform(48, 60)
  lon = -random.uniform(130, 115)
  date_yyyy = random.randint(1970, 2020)
  date_mm = random.randint(1,12)

  if (date_mm <= 3 or date_mm == 5 or date_mm == 7 or date_mm == 8 or date_mm == 10 or date_mm == 12):
    if (date_mm == 2):
      date_dd = int(random.randint(1,28))
    else:
      date_dd = int(random.randint(1,31))
  else:
    date_dd = int(random.randint(1,30))
  
  one_entry.append(date_yyyy*10000+date_mm*100+date_dd)
  one_entry.append("No Cause")
  one_entry.append("No Fire")
  one_entry.append(lat)
  one_entry.append(lon)
  one_entry.append(0.0)
  total_nofire_list.append(one_entry)

df2 = pd.DataFrame(total_nofire_list, columns=df.columns)
df_nofire = df_nofire.append(df2)

In [ ]:
df_nofire.to_csv('data_nofire.csv')
df_yesfire.to_csv('data_yesfire.csv')

Data Collection Part 2: Getting Climate Data

In [ ]:
import requests
import pandas as pd

In [ ]:
#These "RAW" urls may require you to get you own "RAW" url because they stop working after some time (weird github thing)
url_yes = 'https://raw.githubusercontent.com/UBC-CPEN291/project-team-corythosaurus/main/data/data_yesfire.csv?token=AQTQKXO564ZNCWKO7L5FVODATOEM2'
df_yesfire = pd.read_csv(url_yes, index_col=0)
#These "RAW" urls may require you to get you own "RAW" url because they stop working after some time (weird github thing)
url_no = 'https://raw.githubusercontent.com/UBC-CPEN291/project-team-corythosaurus/main/data/data_nofire.csv?token=AQTQKXMYF2ZTDLZBJUEJI3TATOEOW'
df_nofire = pd.read_csv(url_no, index_col=0)

In [ ]:
df_yesfire

,IGN_DATE,FIRE_CAUSE,FIRE_TYPE,LATITUDE,LONGITUDE,SIZE_HA
FIRE_NO,,,,,,
C30085,2.015052e+07,Lightning,Fire,52.2711,-121.5118,3.6
K71366,2.017073e+07,Person,Fire,50.3299,-121.3953,30.0
N20709,2.000081e+07,Person,Fire,49.8731,-115.6792,3.5
G90279,2.010071e+07,Lightning,Fire,58.8575,-121.9688,4.7
G40080,2.000051e+07,Person,Fire,53.9200,-124.0817,25.0
...,...,...,...,...,...,...
G70024,2.010041e+07,Person,Fire,56.0713,-121.8325,2.8
G41084,2.019072e+07,Lightning,Fire,54.0024,-124.6282,2.0
C20014,2.016041e+07,Person,Fire,51.5929,-123.0197,17.0


In [ ]:
df_nofire

,IGN_DATE,FIRE_CAUSE,FIRE_TYPE,LATITUDE,LONGITUDE,SIZE_HA
V81229,2.009080e+07,Person,Nuisance Fire,49.642200,-125.055800,0.0
V10028,2.013052e+07,Person,Nuisance Fire,49.121100,-121.591700,0.0
V10055,2.011052e+07,Person,Nuisance Fire,49.077200,-121.707800,0.0
V70761,2.008101e+07,Person,Nuisance Fire,49.341200,-124.420900,0.0
V60046,2.004043e+07,Unknown,Nuisance Fire,48.783300,-123.950000,0.0
...,...,...,...,...,...,...
2495,1.971071e+07,No Cause,No Fire,54.524433,-121.838602,0.0
2496,1.999021e+07,No Cause,No Fire,48.405481,-116.337460,0.0
2497,1.994050e+07,No Cause,No Fire,51.126953,-127.393534,0.0
2498,1.977022e+07,No Cause,No Fire,59.242536,-125.893874,0.0


In [ ]:
def split(word):
  return [char for char in word]

#To get historical climate data given date, longitude and latitude from visual-crossing rapidapi (Used to add additional features for training/test data)
def get_climate_data(ign_date, latitude, longitude):

  url = "https://visual-crossing-weather.p.rapidapi.com/history"

  ign_date_list = split(str(ign_date))
  yyyy = ''.join(ign_date_list[0:4])
  mm = ''.join(ign_date_list[4:6])
  dd = ''.join(ign_date_list[6:8])
  start_dt = f'{yyyy}-{mm}-{dd}T00:00:00'

  coords = f'{latitude}, {longitude}'

  querystring = {"startDateTime":start_dt,"aggregateHours":"24","location":coords,"endDateTime":start_dt,"unitGroup":"us","dayStartTime":"8:00:00","contentType":"csv","dayEndTime":"17:00:00","shortColumnNames":"0"}

  headers = {
      'x-rapidapi-key': "5416e60fd3msh25313be4cf91e39p1db2f2jsn7acc7269aacf",
      'x-rapidapi-host': "visual-crossing-weather.p.rapidapi.com"
      }

  response = requests.request("GET", url, headers=headers, params=querystring)
  response_list = response.text.split(",")

  return response_list
  # f.write(response.text)

#To get forecasted climate data given longitude and latitude from visual-crossing rapidapi (Used in the finished website to get necessary features, given a user input, to feed into finished model)
def get_future_climate_data(latitude, longitude):
  url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
  coords = f'{latitude}, {longitude}'
  querystring = {"location":coords,"aggregateHours":"24","shortColumnNames":"0","unitGroup":"us","contentType":"csv"}

  headers = {
      'x-rapidapi-key': "5416e60fd3msh25313be4cf91e39p1db2f2jsn7acc7269aacf",
      'x-rapidapi-host': "visual-crossing-weather.p.rapidapi.com"
      }

  response = requests.request("GET", url, headers=headers, params=querystring)
  response_list = response.text.split(",")

  return response_list

In [ ]:
get_climate_data(20030705.09, 49.0707, -121.7133)

['Address',
 'Date time',
 'Minimum Temperature',
 'Maximum Temperature',
 'Temperature',
 'Dew Point',
 'Relative Humidity',
 'Heat Index',
 'Wind Speed',
 'Wind Gust',
 'Wind Direction',
 'Wind Chill',
 'Precipitation',
 'Precipitation Cover',
 'Snow Depth',
 'Visibility',
 'Cloud Cover',
 'Sea Level Pressure',
 'Weather Type',
 'Latitude',
 'Longitude',
 'Resolved Address',
 'Name',
 'Info',
 'Conditions\n"49.0707',
 ' -121.7133"',
 '"07/05/2003"',
 '54.8',
 '71.2',
 '62.2',
 '56.7',
 '83.42',
 '',
 '6.3',
 '',
 '215.87',
 '',
 '0.02',
 '4.17',
 '',
 '8.7',
 '10.7',
 '1018.8',
 '"Mist',
 ' Heavy Rain',
 ' Light Rain"',
 '49.0707',
 '-121.7133',
 '"49.0707',
 ' -121.7133"',
 '"49.0707',
 ' -121.7133"',
 '""',
 '"Rain"\n']

In [ ]:
get_future_climate_data(1,32)

['Address',
 'Date time',
 'Latitude',
 'Longitude',
 'Resolved Address',
 'Name',
 'Wind Direction',
 'Minimum Temperature',
 'Maximum Temperature',
 'Temperature',
 'Wind Speed',
 'Cloud Cover',
 'Heat Index',
 'Chance Precipitation (%)',
 'Precipitation',
 'Sea Level Pressure',
 'Snow Depth',
 'Snow',
 'Relative Humidity',
 'Wind Gust',
 'Wind Chill',
 'Conditions\n"1',
 ' 32"',
 '"05/12/2021"',
 '1.0',
 '32.0',
 '"1',
 ' 32"',
 '"1',
 ' 32"',
 '152.3',
 '63.4',
 '74.9',
 '69.1',
 '2.7',
 '65.7',
 '',
 '57.0',
 '0.15',
 '1014.0',
 '0.0',
 '0.0',
 '89.9',
 '7.6',
 '',
 '"Rain',
 ' Partially cloudy"\n"1',
 ' 32"',
 '"05/13/2021"',
 '1.0',
 '32.0',
 '"1',
 ' 32"',
 '"1',
 ' 32"',
 '178.6',
 '60.9',
 '82.8',
 '68.7',
 '2.7',
 '62.7',
 '83.6',
 '90.5',
 '0.19',
 '1013.3',
 '0.0',
 '0.0',
 '84.2',
 '19.0',
 '',
 '"Rain',
 ' Partially cloudy"\n"1',
 ' 32"',
 '"05/14/2021"',
 '1.0',
 '32.0',
 '"1',
 ' 32"',
 '"1',
 ' 32"',
 '179.7',
 '63.1',
 '80.9',
 '69.3',
 '1.7',
 '75.8',
 '83.2',
 '76.

In [ ]:
tomorrow_climate_data = get_future_climate_data(0,0)[0:46]
tomorrow_climate_data

['Address',
 'Date time',
 'Latitude',
 'Longitude',
 'Resolved Address',
 'Name',
 'Wind Direction',
 'Minimum Temperature',
 'Maximum Temperature',
 'Temperature',
 'Wind Speed',
 'Cloud Cover',
 'Heat Index',
 'Chance Precipitation (%)',
 'Precipitation',
 'Sea Level Pressure',
 'Snow Depth',
 'Snow',
 'Relative Humidity',
 'Wind Gust',
 'Wind Chill',
 'Conditions\n"0',
 ' 0"',
 '"05/12/2021"',
 '0.0',
 '0.0',
 '"0',
 ' 0"',
 '"0',
 ' 0"',
 '167.5',
 '81.9',
 '83.0',
 '82.5',
 '9.4',
 '11.8',
 '89.9',
 '28.6',
 '0.0',
 '1011.7',
 '',
 '0.0',
 '76.4',
 '11.0',
 '',
 '"Clear"\n"0']

In [ ]:
#Reformat fire data to include additional climate data obtained from rapidAPI (For the purpose of training the model)
def get_climate_df(df):
  total_df = []
  date = []
  min_tmp = []
  max_tmp = []
  avg_tmp = []
  dew_point = []
  relative_humidity = []
  heat_index = []
  wind_speed = []
  wind_gust = []
  wind_dir = []
  wind_chill = []
  precipitation = []
  visibility = []
  cloud_cover = []
  sea_level_pressure = []
  weather_type = []
  print("the lenght of the df passed into this function is", len(df))
  
  for i in range (len(df)): 
    response_list = get_climate_data(df['IGN_DATE'][i], df['LATITUDE'][i], df['LONGITUDE'][i])
    
    if (len(response_list) < 22):
      response_list = ['N/A'] * 52
    date.append(response_list[26])  #date, to ensure we are obtaining the right data
    min_tmp.append(response_list[27]) #minimum temperature
    max_tmp.append(response_list[28]) #max
    avg_tmp.append(response_list[29]) #average
    dew_point.append(response_list[30]) #dew point
    relative_humidity.append(response_list[31]) #relative humidity
    heat_index.append(response_list[32]) #heat index
    wind_speed.append(response_list[33]) #wind speed
    wind_gust.append(response_list[34]) #wind gust
    wind_dir.append(response_list[35]) #wind dir
    wind_chill.append(response_list[36]) #wind chill
    precipitation.append(response_list[37]) #precipitation
    visibility.append(response_list[40]) #visibility
    cloud_cover.append(response_list[41]) #cloud cover
    sea_level_pressure.append(response_list[42]) #sea level pressure
    weather_type.append(response_list[43:]) #rest of the response

  df['minimum temperature'] = min_tmp
  df['maximum temperature'] = max_tmp
  df['average temperature'] = avg_tmp
  df['dew point'] = dew_point
  df['relative humidity'] = relative_humidity
  df['heat index'] = heat_index
  df['wind speed'] = wind_speed
  df['wind gust'] = wind_gust
  df['wind direction'] = wind_dir
  df['wind chill'] = wind_chill
  df['precipitation'] = precipitation
  df['visibility'] = visibility
  df['cloud cover'] = cloud_cover
  df['sea level pressure'] = sea_level_pressure
  df['weather_type'] = weather_type

In [ ]:
#Limit of api with free x-rapidapi-key is 500 requests, since we needed to process 10000 data, we needed to make 20 separate accounts to
#get 20 free x-rapidapi-key to process everything. This grueling process is not done only using the two lines below but also through some Excel work

#In any case, NEVER RUN THESE TWO LINES OF CODE AGAIN, which is why we are commenting it, as it will exhaust our current working key which is going to be used for the website

# get_climate_df(df_yesfire)
# get_climate_df(df_nofire)

the lenght of the df passed into this function is 4982
the lenght of the df passed into this function is 5000


In [ ]:
df_yesfire.to_csv('yesfire_complete_data2.csv')

In [ ]:
df_nofire.to_csv('nofire_complete_data2.csv')

Part 2.1: Manual Dataset Manipulation

We combine the two datasets above to a single .csv file. We later observed that only using the randomly generated fires labeled as size 0 produced better results. Hence, the final file used for subsequent sections consist of:
  1. 2500 entries from yesfire_complete_data2
  2. 2500 entries from nofire_complete_data2 - the *randomly* generated entries only.

Part 3: Spliting Dataset for Testing and Training

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

In [ ]:
#These "RAW" urls may require you to get you own "RAW" url because they stop working after some time (weird github thing)
url = 'https://raw.githubusercontent.com/UBC-CPEN291/project-team-corythosaurus/main/data/test_data_milestone3.csv?token=ANIPG2KIV7PYSU63C2CCHBDATSF32'
df_ = pd.read_csv(url, low_memory=False)
df_

,SIZE_HA,IGN_DATE,LATITUDE,LONGITUDE,minimum temperature,maximum temperature,average temperature,dew point,relative humidity,heat index,wind speed,wind gust,wind direction,wind chill,precipitation,visibility,cloud cover,sea level pressure,Unnamed: 18,Unnamed: 19
0,0.0,20140607.12,49.776300,-123.106700,51.8,72.8,62.5,50.5,67.49,NaN,10.2,20.8,193.71,NaN,0.01,633.7,0.0,1019.7,NaN,NaN
1,0.0,20130820.11,49.194400,-125.391900,50.3,73.5,62.4,49.1,64.63,NaN,7.8,NaN,271.46,NaN,0.00,99.0,5.9,1020.7,NaN,NaN
2,0.0,19920730.00,56.646412,-129.772666,46.5,71.7,60.9,49.6,70.22,NaN,11.4,NaN,188.57,NaN,0.00,19.5,45.8,1019.9,NaN,NaN
3,0.0,20170131.00,54.956750,-126.093563,-4.6,23.6,7.9,2.6,79.90,NaN,5.9,NaN,235.29,-0.3,0.06,96.2,3.1,1037.9,NaN,NaN
4,0.0,19781008.00,51.313833,-122.866839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,165.0,20140728.15,50.343400,-126.882200,53.8,66.8,59.0,55.8,89.73,NaN,12.4,20.8,111.38,NaN,0.00,32.3,22.4,1020.7,NaN,NaN
4994,2392.0,20180811.16,49.072300,-115.457100,61.5,93.8,78.7,42.3,28.50,90.1,12.2,26.0,182.75,NaN,0.00,91.8,10.4,1008.3,NaN,NaN
4995,19226.0,20180717.17,49.084600,-119.869500,69.4,96.9,83.0,44.8,27.67,92.4,9.2,25.3,199.54,NaN,0.00,628.4,0.0,1011.2,NaN,NaN
4996,4.0,20030731.16,48.355000,-123.575000,55.0,67.3,60.2,55.8,85.95,NaN,19.4,NaN,223.71,NaN,0.00,NaN,0.0,1014.2,NaN,NaN


In [ ]:
#Cleanup data
def cleanup_data(df):
  #if it is a string (aka. 'N/A'), fill with nan. See part 1 for details
  def to_nan(entry):
    if type(entry) is not str: return entry
    else: return np.nan
  
  #fixing the missing data

  for column in df.columns:
    df[column] = df[column].apply(to_nan)  
    avg = df[column].mean()
    print(avg)
    df[column].fillna(avg, inplace=True)    #Replace with average

   #fixing the missing data, end
   

  df['SIZE_HA'] = df['SIZE_HA'].values + 1 #Because log(0) is bad

  df.drop(columns=['Unnamed: 18'], inplace = True)
  df.drop(columns=['Unnamed: 19'], inplace = True)

  #eliminating outliers
  q25, q75 = np.percentile(df['SIZE_HA'], 25), np.percentile(df['SIZE_HA'], 75)
  iqr = q75 - q25
  cut_off = iqr * 1.5
  lower, upper = q25 - cut_off, q75 + cut_off
  df= df[df['SIZE_HA'] <= upper]
  #eliminating outliers, end


  return df

In [ ]:
#Cleanup dataset
df_ = cleanup_data(df_)

330.48600520208055
20026124.90970577
53.40442305249716
-122.14423555004001
40.58008083140884
63.09509237875295
52.084209006928425
37.187727006444
62.41874633860559
85.18183962264142
12.53463329452857
27.2919663351186
192.44023689197786
27.720631578947398
0.06612954545454518
76.20172189733584
30.337810514153652
1015.3026817640034
nan
nan


In [ ]:
#Split dataset to train and test sets
df_train = df_[df_['IGN_DATE'] < 20150000]
df_test = df_[df_['IGN_DATE'] >= 20150000]
print (df_train)
print (df_test)

      SIZE_HA     IGN_DATE  ...  cloud cover  sea level pressure
0         1.0  20140607.12  ...     0.000000         1019.700000
1         1.0  20130820.11  ...     5.900000         1020.700000
2         1.0  19920730.00  ...    45.800000         1019.900000
4         1.0  19781008.00  ...    30.337811         1015.302682
6         1.0  19911202.00  ...    30.337811         1015.302682
...       ...          ...  ...          ...                 ...
4988      6.1  20100621.18  ...    93.300000         1015.900000
4990      8.4  20040624.16  ...    18.100000         1015.800000
4992     13.0  20060723.23  ...     7.700000         1010.800000
4996      5.0  20030731.16  ...     0.000000         1014.200000
4997      9.0  20010515.15  ...     0.000000         1006.200000

[3406 rows x 18 columns]
      SIZE_HA     IGN_DATE  ...  cloud cover  sea level pressure
3         1.0  20170131.00  ...     3.100000         1037.900000
5         1.0  20170525.00  ...    49.000000         1007.500000

In [ ]:
#Split both train and test datasets into inputs and outputs
sizeHA_train = np.log(df_train['SIZE_HA'].values)
sizeHA_test = np.log(df_test['SIZE_HA'].values)
feats_train = df_train.drop(columns=['SIZE_HA'])
feats_test = df_test.drop(columns=['SIZE_HA'])

Part 4: Random Forests

In [ ]:
!pip install scikit-learn==0.24.1
import datetime, re, math
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy import std
import sklearn, sklearn.tree, sklearn.ensemble, sklearn.feature_extraction, sklearn.metrics

     |████████████████████████████████| 22.3MB 4.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
#Getting features
dvr = sklearn.feature_extraction.DictVectorizer()
enc_feats_train = dvr.fit_transform(feats_train.T.to_dict().values())
enc_feats_test = dvr.transform(feats_test.T.to_dict().values())

In [ ]:
dvr.get_feature_names()

['IGN_DATE',
 'LATITUDE',
 'LONGITUDE',
 'average temperature',
 'cloud cover',
 'dew point',
 'heat index',
 'maximum temperature',
 'minimum temperature',
 'precipitation',
 'relative humidity',
 'sea level pressure',
 'visibility',
 'wind chill',
 'wind direction',
 'wind gust',
 'wind speed']

In [ ]:
#To evaluate how good RF model is 
def eval_model(model):
    preds_train = model.predict(enc_feats_train)
    preds_test = model.predict(enc_feats_test)
    for i in range (len(preds_test)):
      print(f"the labels: {sizeHA_test[i]}: the prediction {preds_test[i]}")
    tbl = [[sklearn.metrics.mean_absolute_error(*vecs),
            sklearn.metrics.median_absolute_error(*vecs),
            math.sqrt(sklearn.metrics.mean_squared_error(*vecs))]
           for vecs in [(sizeHA_train, preds_train), (sizeHA_test, preds_test)]]
    df = pd.DataFrame(tbl, columns=['MAE','MdAE', 'RMSE'], index=['training', 'testing'])
    return df.T.round(2)

In [ ]:
# Random forests!!!
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=80, max_samples=3406, max_features=0.7, min_samples_leaf=4, random_state=291, n_jobs=-1)
rf.fit(enc_feats_train, sizeHA_train)
# Testing Random Forest
eval_model(rf)

the labels: 0.0: the prediction 0.23997708619427294
the labels: 0.0: the prediction 1.8812911752300774
the labels: 0.0: the prediction 0.24498561887859632
the labels: 0.0: the prediction 0.5330358457990255
the labels: 0.0: the prediction 1.1106028984859613
the labels: 0.0: the prediction 1.214884089641989
the labels: 0.0: the prediction 1.8322745267787997
the labels: 0.0: the prediction 0.04143006544828498
the labels: 0.0: the prediction 1.3023289549816988
the labels: 0.0: the prediction 0.9823263073091228
the labels: 0.0: the prediction 1.0338428404674862
the labels: 0.0: the prediction 0.7674760381332186
the labels: 0.0: the prediction 0.14933955779457658
the labels: 0.0: the prediction 1.43211192002276
the labels: 0.0: the prediction 1.012474401527372
the labels: 0.0: the prediction 0.1750600858299724
the labels: 0.0: the prediction 1.3603556447141005
the labels: 0.0: the prediction 0.8579518801871219
the labels: 0.0: the prediction 0.25676819553908575
the labels: 0.0: the predictio

,training,testing
MAE,0.22,0.70
MdAE,0.05,0.59
RMSE,0.39,0.89


Part 5: Multiple Regression Model


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn import linear_model
import math
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


In [ ]:
#This is a multiple Regression Model
model = linear_model.LinearRegression()
model.fit(feats_train, sizeHA_train)

df_test = pd.DataFrame(sizeHA_test)
print(df_test)

            0
0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.000000
..        ...
713  1.945910
714  1.435085
715  1.098612
716  1.931521
717  1.386294

[718 rows x 1 columns]


In [ ]:
# Testing Multiple Regression Model

def eval_lr(model):
  preds_train = model.predict(feats_train)
  preds_test = model.predict(feats_test) 
  tbl = [[sklearn.metrics.mean_absolute_error(*vecs),
            sklearn.metrics.median_absolute_error(*vecs),
            math.sqrt(sklearn.metrics.mean_squared_error(*vecs))]
           for vecs in [(sizeHA_train, preds_train), (sizeHA_test, preds_test)]]
  df = pd.DataFrame(tbl, columns=['MAE','MdAE', 'RMSE'], index=['training', 'testing'])
  return df.T.round(2)
#Computing the loss for half the data in the dataset

In [ ]:
eval_lr(model)

,training,testing
MAE,0.57,0.85
MdAE,0.44,0.82
RMSE,0.75,1.00


Part 6: the ensemble

In [ ]:
def eval_ensemble(rf_model, mlp_model, entries, display_ensemble_loss):
  #evaluate the performance of the ensemble
  #entries must be in the form of: IGN_DATE	LATITUDE	LONGITUDE	minimum temperature	maximum temperature	average temperature	dew point	relative humidity	heat index	wind speed	wind gust	wind direction	wind chill	precipitation	visibility	cloud cover	sea level pressure
  rf_entries = dvr.fit_transform(entries.T.to_dict().values()) 
  rf_preds = rf_model.predict(rf_entries)
  mlp_preds = mlp_model.predict(entries)
  combined_preds = [] #return predicted size.
  
  for i in range (len(entries)):
    combined_prediction = (rf_preds[i] + mlp_preds[i]) / 2.0            
    combined_preds.append(combined_prediction)
  
  for i in range (len(entries)):
    print("[" , i , "]: size_label:", sizeHA_test[i], " rf_preds ", rf_preds[i], "mlp_preds: ", mlp_preds[i], " combined_preds: ", combined_preds[i])

  if (display_ensemble_loss):
    tbl = [[sklearn.metrics.mean_absolute_error(*vecs),
            sklearn.metrics.median_absolute_error(*vecs),
            math.sqrt(sklearn.metrics.mean_squared_error(*vecs))]
           for vecs in [(sizeHA_test, combined_preds)]]                         
    df = pd.DataFrame(tbl, columns=['MAE','MdAE', 'RMSE'], index=['testing'])
    return df.T.round(2)

  return combined_preds 


In [ ]:
final_preds = eval_ensemble(rf, model, feats_test, 1)

[ 0 ]: size_label: 0.0  rf_preds  0.23997708619427294 mlp_preds:  0.87683074252773  combined_preds:  0.5584039143610015
[ 1 ]: size_label: 0.0  rf_preds  1.8812911752300774 mlp_preds:  1.6264277192288574  combined_preds:  1.7538594472294675
[ 2 ]: size_label: 0.0  rf_preds  0.24498561887859632 mlp_preds:  0.9739518154141393  combined_preds:  0.6094687171463679
[ 3 ]: size_label: 0.0  rf_preds  0.5330358457990255 mlp_preds:  0.23316299389217932  combined_preds:  0.3830994198456024
[ 4 ]: size_label: 0.0  rf_preds  1.1106028984859613 mlp_preds:  1.207563262957109  combined_preds:  1.159083080721535
[ 5 ]: size_label: 0.0  rf_preds  1.214884089641989 mlp_preds:  1.3042359450069654  combined_preds:  1.2595600173244772
[ 6 ]: size_label: 0.0  rf_preds  1.8322745267787997 mlp_preds:  1.462928820594442  combined_preds:  1.6476016736866208
[ 7 ]: size_label: 0.0  rf_preds  0.04143006544828498 mlp_preds:  1.1386176573889628  combined_preds:  0.590023861418624
[ 8 ]: size_label: 0.0  rf_preds  1

In [ ]:
print(final_preds)
#...

      testing
MAE      0.77
MdAE     0.70
RMSE     0.91
